# exemple faiss

pip install faiss-gpu

In [ ]:
pip list

Package                           Version
--------------------------------- --------------------
absl-py                           1.0.0
accelerate                        0.5.1
addict                            2.4.0
aiohttp                           3.8.1
aiosignal                         1.2.0
alabaster                         0.7.12
albumentations                    1.3.0
allennlp                          2.9.0
antlr4-python3-runtime            4.8
anyio                             3.5.0
apex                              0.1
appdirs                           1.4.4
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.2
assertpy                          1.1
astroid                           2.9.3
astropy                           5.0.1
asttokens                         2.0.5
astunparse                        1.6.3
async-timeout                     4.0.2
atomicwrites                      1.4.0
attrs                     

In [4]:
import faiss
import numpy as np

# Define parameters
d = 256      # Vector dimension (bits)
N = 100000   # Number of binary vectors

# Generate random binary dataset (values are 0 or 1)
data = np.random.randint(2, size=(N, d // 8), dtype='uint8')  # Store as bytes

# Create a binary flat index
index = faiss.IndexBinaryFlat(d)

# Add data to the index
index.add(data)

# Query a random binary vector
query = np.random.randint(2, size=(1, d // 8), dtype='uint8')

# Perform search
D, I = index.search(query, 5)  # Find 5 nearest neighbors

print("Nearest neighbors' indices:", I)
print("Hamming distances:", D)


Nearest neighbors' indices: [[40007 18418 20088 67026 77215]]
Hamming distances: [[4 5 5 5 5]]


In [3]:
import faiss
import numpy as np

# Generate random 256-D vectors
N = 100000  # Number of vectors
D = 256  # Dimension
data = np.random.random((N, D)).astype('float32')

# Create an LSH index
num_bits = 16  # Number of bits for hashing
index = faiss.IndexLSH(D, num_bits)

# https://github.com/facebookresearch/faiss/wiki/Faiss-indexes/1f721de164ae7950c65914d2dde0865312584a36


# Add vectors to the index
index.add(data)

# Query
#query = np.random.random((1, D)).astype('float32')
query = np.expand_dims(data[100], 0)
D, I = index.search(query, 10)  # Find top-10 nearest neighbors

print("Nearest neighbors:", I)


TypeError: Input argument codes must be ndarray of dtype  uint8, but found float32

In [3]:
from scipy.spatial.distance import cdist
feat_dists = cdist(query, data, metric='cosine').reshape(-1)
min_dist, nearest_idx = np.min(feat_dists), np.argmin(feat_dists)

In [19]:
[  cdist(np.expand_dims(data[x], 0)[0], data, metric='cosine').reshape(-1) for x in I]

[array([0.23543808, 0.24439688, 0.25769359, ..., 0.24738973, 0.29674815,
        0.24196164])]

In [16]:
len([ np.expand_dims(data[x], 0) for x in I][0])

1

# faiss sur kitti

In [43]:
import json
import os
import torch
from module_loader_kitti_pose import * # add for more metrics
import faiss
eval_seq = 22
num_bits = 32 

log3dnet_dir=os.getenv('LOG3DNET_DIR')
## ==== Kitti =====
WORK_PATH = "/lustre/fsn1/worksf/projects/rech/dki/ujo91el"
kitti_dir = WORK_PATH+"/datas/datasets/"
eval_seq = '%02d' % eval_seq
sequence_path = kitti_dir + 'sequences/' + eval_seq + '/'
tfs, pose = load_poses_from_txt(sequence_path + 'poses.txt')
num_queries = len(pose)
print("num_queries ",num_queries)
embeddings = []
for query_idx in range(num_queries):
    padded_string = str(query_idx).zfill(6)            
    log_desc = sequence_path + "/logg_desc/" + padded_string + '.pt'
    xx = torch.load(log_desc)
    embeddings.append(xx)

data = np.stack([t.detach().cpu().numpy() for t in embeddings])
D = 256  # Dimension
# Create an LSH index
 # Number of bits for hashing
index = faiss.IndexLSH(D, num_bits)
# https://github.com/facebookresearch/faiss/wiki/Faiss-indexes/1f721de164ae7950c65914d2dde0865312584a36


# Add vectors to the index
index.add(data)

query = np.expand_dims(data[1], 0)
D, I = index.search(query, 10)  # Find top-10 nearest neighbors
print("Nearest neighbors:", I)

# distance
print("Distance neighbors:",[ round(np.linalg.norm(pose[1]- pose[place]),2) for place in I[0]])

num_queries  18236
Nearest neighbors: [[   1    3 4458 6341    0    9   11 2855 2860 4452]]
Distance neighbors: [0.0, 1.72, 6.88, 2240.56, 0.86, 6.88, 8.6, 508.82, 510.06, 2.55]


In [48]:
D

array([[0., 0., 0., 0., 1., 1., 1., 1., 1., 1.]], dtype=float32)